# Reinforcement predictions

## Initialise code

First, we need to load some packages and functions.

In [ ]:
# load packages and functions
import numpy as np
import matplotlib.pyplot as plt
from pyrootmemo.io import read_csv_roots
from pyrootmemo.fit import Powerlaw
from pyrootmemo.materials import Soil, Interface
from pyrootmemo.geometry import SoilProfile, FailureSurface
from pyrootmemo.models import Wwm, Fbm, Rbmw, Waldron

# define parameter object (value + unit), used later to define soil, root and geometry parameters
from pyrootmemo.helpers import units  # load the unit system used
from collections import namedtuple
Parameter = namedtuple("parameter", "value unit")

## Load data

There are two files to load, one for biomechanical testing results, and describing the diameters of all roots in the experiment used to measure reinforcement. 

By default, this script uses biomechanics data for willow roots as measured by Liang et al. (2017), see https://doi.org/10.1016/j.ecoleng.2017.06.067.

By default, root reinforcements are predicted for an (artificial) set of *three roots only*, to make it easier to see what models predict and why. These diameters are stored in the data file "data/experiment_dummy_data.csv". Once you understand how the script and models function, you can swap out this 'dummy' data for real measured diameter data on soil reinforcement by willow roots, as measured for a direct shear test by Bull et al. (2022), see https://doi.org/10.1098/rspa.2021.0210 (data file 'data/experiment_willowC_bull.csv'). 

You can also experiment with some of the data for *Lolium perenne* and *Onobrychis viciifolia* as discussed earlier during this workhop. When you do this, you will need to update some (estimated or measured) properties such as the root length or the cross-sectional area of the shear plane.

Alternatively, you can read in your own data by placing a comma seperated .csv files in the project folder and adjusting the paths below. In this case:

* The .csv file with 'field' data should contain columns for:
  * `diameter_unit`: the diameter of the roots
* The .csv file describing biomechanical data should contain columns for:
  * `diameter_unit`: the diameters of the tested roots
  * `tensile_strength_unit`: the tensile strengths of the tested roots
  * `elastic_modulus_unit`: the (elastic) stiffness of the tested roots

The part of the column name indicated by 'unit' must be replaced by the measurement unit, e.g. 'mm'.

In [ ]:
# load root diameter data (use "#" at the start of a code line to disable execution of this line)
roots_experiment = read_csv_roots('data/experiment_dummy_data.csv')
# roots_experiment = read_csv_roots('data/experiment_willowC_bull.csv')
# roots_experiment = read_csv_roots('data/experiment_lolium_perenne_talveg_20170615.csv')
# roots_experiment = read_csv_roots('data/experiment_onobrychis_viciifolia_talveg_20170615.csv')

# measure biomechanical data (use "#" at the start of a code line to disable execution of this line)
roots_biomech = read_csv_roots('data/tensile_willow_liang2017.csv') 
# roots_biomech = read_csv_roots('data/tensile_lolium_perenne_talveg.csv') 
# roots_biomech = read_csv_roots('data/tensile_onobrychis_viciifolia_talveg.csv') 

We could compare the differences in diameter distribution between roots tested for biomechanical properties, and those counted in the shear strength test.

In [ ]:
# generate a plot with two subplots
fig, axs = plt.subplots(1, 2, sharex = True, sharey = True)
fig.supxlabel('diameter [mm]')
fig.supylabel('density [-]')

# plot biomechanical in first subplot
axs[0].hist(roots_biomech.diameter.to('mm').magnitude, density = True)
axs[0].title.set_text('Biomechanical testing')

# plot experiment in second subplot
axs[1].hist(roots_experiment.diameter.to('mm').magnitude, density = True)
axs[1].title.set_text('Reinforcement experiment')

# Biomechanics 

First, we need to fit the biomechanical data, so we can predict root biomechanical properties for all roots in the *experiment*.

In [ ]:
# create a power law for tensile strength
fit_tensile_strength = Powerlaw(
    roots_biomech.diameter,
    roots_biomech.tensile_strength,
    model = 'normal'
)

# create a power law for root stiffness
fit_elastic_modulus = Powerlaw(
    roots_biomech.diameter,
    roots_biomech.elastic_modulus,
    model = 'normal'
)

With these fits, we can now predict the strength and stiffness of any root in the experiment

In [ ]:
# predict tensile strenghts
roots_experiment.tensile_strength = fit_tensile_strength.predict(roots_experiment.diameter)
# predict stiffnesses
roots_experiment.elastic_modulus = fit_elastic_modulus.predict(roots_experiment.diameter)

For some models (Waldron, RBMw), we also need to know the (anchored) length of the root. 

Bull et al.'s expriments were sheared half-way a 500 mm long core. Therefore, let's assume that the (anchored) length, i.e. the distance from the shear plane to the tip of all roots, is 250 mm for all roots.

In [ ]:
# diameter--length power law parameters
length_multiplier = 250 * units('mm')
length_exponent = 0.0
reference_diameter = 1.0 * units('mm')

# set length for each root, using power law
roots_experiment.length = (
    length_multiplier 
    * (roots_experiment.diameter / reference_diameter) ** length_exponent
    )

# Set soil and interface properties

Let's define come soil properties. You can define multiple soils, but In Bull's experiment, there is only one soil.

In [ ]:
soil = Soil(
    'silty SAND',
    cohesion = Parameter(0.0, 'kPa'),
    friction_angle = Parameter(36.4, 'degrees')
)

With the soil(s) defined, we can now define a soil profile. This is a list of soils - stacked on top of each other (top to bottom). For each soil in the 'stack' define the bottom of the layer.

Bull's tests were conducted in a 500 mm long core, so the depth of the silty sand is at 500 mm below the soil surface

In [ ]:
soil_profile = SoilProfile(
    [soil],
    depth = Parameter([500.0], 'mm')
)

The Waldron model requires defining the strength of the root-soil interface.

In [ ]:
interface = Interface(
    shear_strength = Parameter(2.7, 'kPa')
)

Finally, let's set some parameters for the failure surface. In Bull's test, these were located at 250 mm depth. The cores were 110 mm in diameter.

In [ ]:
failure_surface = FailureSurface(
    cross_sectional_area = Parameter(0.25 * np.pi * 110**2, 'mm^2'),
    depth = Parameter(250, 'mm'),
    shear_zone_thickness = Parameter(30, 'mm')
)

## Reinforcement predictions

With all properties defined, we can now start to make reinforcement predictions using the various models.

### WWM

In [ ]:
# create a Wwm model object
wwm = Wwm(roots_experiment)

# calculate and show the peak reinforcement (in kPa)
peak_reinforcement = wwm.peak_reinforcement(failure_surface)
print(peak_reinforcement.to('kPa'))

### FBM

In [ ]:
# define the load sharing parameter
load_sharing = 1.0

# create a Fbm model object
fbm = Fbm(roots_experiment, load_sharing)

# calculate and show the peak reinforcement (in kPa)
peak_reinforcement = fbm.peak_reinforcement(failure_surface, k = 1.2)
print(peak_reinforcement.to('kPa'))

# show a plot demonstrating how the reinforcement mobilises
# if `stack = True`, show all contributions of individual roots (can be labelled, e.g. by diameter)
fbm.plot(
    stack = True, 
    labels = [str(round(d, 2)) for d in roots_experiment.diameter.magnitude]
    )

### RBMw

In [ ]:
# define the Weibull shape parameter
weibull_shape = 4.0

# create a Rbmw model object
rbmw = Rbmw(roots_experiment, weibull_shape)

# calculate and show the peak reinforcement (in kPa)
peak = rbmw.peak_reinforcement(failure_surface)
print(peak['reinforcement'].to('kPa'))

# show a plot demonstrating how the force in the bundle mobilises with displacement
# if `stack = True`, show all contributions of individual roots (can be labelled, e.g. by diameter)
rbmw.plot(
    stack = True,
    labels = [str(round(d, 2)) for d in roots_experiment.diameter.magnitude]
    )

### Waldron

In [ ]:
# create a Waldron model object. We can include the effects of breakage and/or 
# slippage by setting model parameters to `True` or `False`
waldron = Waldron(
    roots_experiment,
    interface,
    soil_profile,
    failure_surface,
    breakage = True,   # allow for breaking of roots (True) or no breakage (False)
    slipping = True    # allow for root slippage (True) or not (False)
    )

# calculate and show the peak reinforcement (in kPa)
peak = waldron.peak_reinforcement()
print('peak reinforcement:', peak['reinforcement'].to('kPa'))
print('peak displacement:', peak['displacement'])

# show how the reinforcement mobilises with soil shear displacement
# if `stack = True`, show all contributions of individual roots (can be labelled, e.g. by diameter)
waldron.plot(
    stack = True,
    labels = [str(round(d, 2)) for d in roots_experiment.diameter.magnitude],
    margin_axis = 0.5  # plot some extra x-axis range, as fraction of default range
    )

## Explore!

You can now investigate:

* What happens when you use realistic, i.e. measured, diameter data?

* How do the different reinforcement models compare? For example, in terms of:
  
  * Peak reinforcement (root 'cohesion')
  * Displacements required to reach peak (Waldron, Rbmw)
  * Which roots are predicted to break first - thicker or thinner ones?

* What happens if you make different assumptions, for example for:
  1. Power law fitting methods for root tensile strength and/or root stiffness
  2. FBM load sharing paramter
  3. RBMw Weibull shape parameter
  4. Different soil properties (friction angle)
  5. Different soil-root interface properties (in Waldron model). When do roots slip? When to roots break? Is accounting for slippage important?
  6. Different failure surface properties (e.g. shear zone thickness in Waldron's model?)

* Try a different dataset, e.g. for:

  - *Lolium perenne*
  - *Onobrychis viciifolia*

  You may need to update assuptions for root length and/or cross-sectional area of the shear plane. How do the results change, and how are the root reinforcement models differ?